In [1]:
# Importing Necessary libraries
import re
import urllib,requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import sqlite3


url = "https://en.wikipedia.org/wiki/September_10"


# Parsing the html content using Beautiful Soup
page = requests.get(url) 
data = BeautifulSoup(page.content,'html.parser')
# Getting list of all the <ul> tags from the html source code
data1 = data.find_all("ul")


In [2]:
# Creating a connetion to a Database
sqliteConnection = sqlite3.connect('SQLite_Python.db')

 
# Creating a table with the required columns
sqlite_create_table_query = '''CREATE TABLE Wikidata (
                                Date TEXT,
                                Description TEXT,
                                Type TEXT);'''
# Initializing a cursor for Database connection
cursor = sqliteConnection.cursor()
# Executing the cursor to create the table
cursor.execute(sqlite_create_table_query)
# Executing the Commit function
sqliteConnection.commit()
print('Wikidata table has created....')

Wikidata table has created....


In [3]:
for j in range(1,4):
    lis = data1[j].find_all('li') 
    for i in lis:
# Splitting the text into dates and descriptions based on "-"
        c,d = re.sub("<.*?>","",str(i)).split("–")
        if j==1:
            type1 = "Events"
        elif j==2:
            type1 = "Births"
        else:
            type1 = "Deaths"
# Inserting our Split data into required columns
        sqlite_insert_query = """INSERT INTO Wikidata
                          (Date, Description, Type) 
                           VALUES 
                          (?,?,?);"""
        data_tuple = (c,d,type1)
        cursor.execute(sqlite_insert_query, data_tuple)

    #Performing a commit on our insert operations
        sqliteConnection.commit()

In [4]:
# Queriying our table Database
sqlite_print_query = """select * from  Wikidata"""
cursor.execute(sqlite_print_query)
rows = cursor.fetchall()

for row in rows:
    print(row)

('506 ', ' The bishops of Visigothic Gaul meet in the Council of Agde.', 'Events')
('1419 ', ' John the Fearless, Duke of Burgundy is assassinated by adherents of the Dauphin, the future Charles VII of France.', 'Events')
('1509 ', ' An earthquake known as "The Lesser Judgment Day" hits Constantinople.', 'Events')
('1515 ', ' Thomas Wolsey is invested as a Cardinal.', 'Events')
('1547 ', ' The Battle of Pinkie, the last full-scale military confrontation between England and Scotland, resulting in a decisive victory for the forces of Edward VI.', 'Events')
('1561 ', ' Fourth Battle of Kawanakajima: Takeda Shingen defeats Uesugi Kenshin in the climax of their ongoing conflicts.', 'Events')
('1570 ', ' Spanish Jesuit missionaries land in present-day Virginia to establish the short-lived Ajacán Mission.', 'Events')
('1573 ', ' German pirate Klein Henszlein and 33 of his crew beheaded in Hamburg [1].', 'Events')
('1608 ', ' John Smith is elected council president of Jamestown, Virginia.', 'E

In [5]:
# Queriying data as per specific case
def query_data(category='Births'):
    sqlite_print_query = f"""select * from  Wikidata"""
    cursor.execute(sqlite_print_query)
    rows = cursor.fetchall()
    rows = [row for row in rows if row[2]==category]
    return rows

In [6]:
# convert queried data into dataframe
import pandas as pd
birth_data = query_data(category='Births')
df = pd.DataFrame(birth_data, columns=['Year', 'Description', 'Type'])
df

,Year,Description,Type
0,877,"Eutychius, patriarch of Alexandria (d. 940)",Births
1,904,"Guo Wei, posthumously known as Emperor Taizu ...",Births
2,920,Louis IV of France (d. 954),Births
3,1169,"Alexios II Komnenos, Byzantine emperor (d. 1183)",Births
4,1423,"Eleanor, Princess of Asturias (d. 1425)[2]",Births
...,...,...,...
227,1992,"Ricky Ledo, American basketball player",Births
228,1992,"Ayub Masika, Kenyan footballer",Births
229,1994,"Mohamed Sylla, French rapper",Births
230,1997,"Brooke Henderson, Canadian golfer",Births
